In [1]:
all_season_links = ["https://fbref.com/en/comps/9/282/2007-2008-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/338/2008-2009-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/400/2009-2010-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/467/2010-2011-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/534/2011-2012-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/602/2012-2013-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/669/2013-2014-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/733/2014-2015-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/1467/2015-2016-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/1526/2016-2017-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/1631/2017-2018-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/1889/2018-2019-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/3232/2019-2020-Premier-League-Stats",
                   "https://fbref.com/en/comps/9/Premier-League-Stats"]

In [2]:
import pandas as pd
from proxy_requests import ProxyRequests
import requests
from bs4 import BeautifulSoup as soup
import time
%run scrape_one_match_code.ipynb

In [14]:
def scrape_all_links(link_list):
    all_dfs = []
    print("{} links to scrape".format(len(link_list)))
    for i in range(len(link_list)):
        link = link_list[i]
        r = ProxyRequests(link)
        r.get()
        page = soup(r.get_raw(), "html.parser")
        df = get_all_match_data(page)
        all_dfs.append(df)
        print("{}/{} complete ({})".format(i+1,len(link_list),link.split("/")[-1]))
    final_df = pd.concat(all_dfs,axis=0)
    return final_df

def check_game_data_available(row):
    match_report = str(row.find("td",{"data-stat":"match_report"}).text)
    if match_report != "Match Report":
        return False
    xg_a = str(row.find("td",{"data-stat":"xg_a"}).text)
    if xg_a == "":
        return False
    return True

def get_all_match_links(page):
    link_list = []
    main_table_body = page.find("tbody")
    data_rows = main_table_body.findAll("tr")
    for row in data_rows:
        isValid = check_game_data_available(row)
        if isValid:
            td = row.find("td",{"data-stat":"match_report"})
            if td.a is not None:
                text = td.a.text
                link = td.a["href"]
                if text == "Match Report":
                    link_list.append("https://fbref.com" + link)
    return link_list

In [15]:
def get_season_data(season_link):
    r = ProxyRequests(season_link)
    r.get()
    page = soup(r.get_raw(), "html.parser")
    link_list = get_all_match_links(page)
    final_df = scrape_all_links(link_list)
    return final_df

In [16]:
t1 = time.time()
final_df = get_season_data("https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures")
t2 = time.time()
print("Took {} seconds".format(round(t2-t1,2)))

125 links to scrape
1/125 complete (Fulham-Arsenal-September-12-2020-Premier-League)
2/125 complete (Crystal-Palace-Southampton-September-12-2020-Premier-League)
3/125 complete (Liverpool-Leeds-United-September-12-2020-Premier-League)
4/125 complete (West-Ham-United-Newcastle-United-September-12-2020-Premier-League)
5/125 complete (West-Bromwich-Albion-Leicester-City-September-13-2020-Premier-League)
6/125 complete (Tottenham-Hotspur-Everton-September-13-2020-Premier-League)
7/125 complete (Sheffield-United-Wolverhampton-Wanderers-September-14-2020-Premier-League)
8/125 complete (Brighton-and-Hove-Albion-Chelsea-September-14-2020-Premier-League)
9/125 complete (Everton-West-Bromwich-Albion-September-19-2020-Premier-League)
10/125 complete (Leeds-United-Fulham-September-19-2020-Premier-League)
11/125 complete (Manchester-United-Crystal-Palace-September-19-2020-Premier-League)
12/125 complete (Arsenal-West-Ham-United-September-19-2020-Premier-League)
13/125 complete (Southampton-Tottenha

106/125 complete (Liverpool-Wolverhampton-Wanderers-December-6-2020-Premier-League)
107/125 complete (Brighton-and-Hove-Albion-Southampton-December-7-2020-Premier-League)
108/125 complete (Leeds-United-West-Ham-United-December-11-2020-Premier-League)
109/125 complete (Wolverhampton-Wanderers-Aston-Villa-December-12-2020-Premier-League)
110/125 complete (Newcastle-United-West-Bromwich-Albion-December-12-2020-Premier-League)
111/125 complete (Manchester-Derby-Manchester-United-Manchester-City-December-12-2020-Premier-League)
112/125 complete (Everton-Chelsea-December-12-2020-Premier-League)
113/125 complete (Southampton-Sheffield-United-December-13-2020-Premier-League)
114/125 complete (Crystal-Palace-Tottenham-Hotspur-December-13-2020-Premier-League)
115/125 complete (Fulham-Liverpool-December-13-2020-Premier-League)
116/125 complete (Arsenal-Burnley-December-13-2020-Premier-League)
117/125 complete (Leicester-City-Brighton-and-Hove-Albion-December-13-2020-Premier-League)
118/125 comple

In [17]:
print(final_df.shape)
final_df.head()

(3415, 128)


,name,pos,age,minutes,goals,assists,pens_made,pens_att,shots_total,shots_on_target,...,away_name,away_score,away_xg,away_manager,date,time,matchweek,stadium,ref,var_ref
0,Aboubakar Kamara,FW,25,62,0,0,0,0,1,0,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote
1,Aleksandar Mitrović,FW,25,28,0,0,0,0,1,1,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote
2,Ivan Cavaleiro,LW,26,90,0,0,0,0,0,0,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote
3,Neeskens Kebano,RW,28,62,0,0,0,0,1,0,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote
4,Andre-Frank Zambo Anguissa,RW,24,28,0,0,0,0,0,0,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote


In [18]:
final_df.to_csv("181220_all_player_data_for_2021_season.csv",index=False)

In [20]:
final_df.to_csv("C:/Users/joeco/Python/fb-ref-data/data/181220_2021_season_player_data.csv",index=False)

In [31]:
final_df[["home_name","away_name"]].groupby(["home_name","away_name"]).size()

array([26, 27, 28, 24, 25], dtype=int64)

In [36]:
list(set(final_df.set_index(['home_name','away_name']).index.tolist()))

[('Liverpool', 'Tottenham Hotspur'),
 ('Aston Villa', 'Sheffield United'),
 ('Leicester City', 'Burnley'),
 ('Manchester City', 'Fulham'),
 ('Everton', 'West Bromwich Albion'),
 ('Burnley', 'Everton'),
 ('Leeds United', 'Leicester City'),
 ('Liverpool', 'Sheffield United'),
 ('Sheffield United', 'West Ham United'),
 ('Manchester City', 'Arsenal'),
 ('Crystal Palace', 'Everton'),
 ('Wolverhampton Wanderers', 'Newcastle United'),
 ('West Bromwich Albion', 'Chelsea'),
 ('Arsenal', 'Aston Villa'),
 ('Crystal Palace', 'Brighton & Hove Albion'),
 ('Brighton & Hove Albion', 'Liverpool'),
 ('West Ham United', 'Newcastle United'),
 ('Everton', 'Liverpool'),
 ('Manchester United', 'Manchester City'),
 ('Everton', 'Brighton & Hove Albion'),
 ('Brighton & Hove Albion', 'Manchester United'),
 ('Southampton', 'Tottenham Hotspur'),
 ('Fulham', 'Arsenal'),
 ('Everton', 'Manchester United'),
 ('Brighton & Hove Albion', 'West Bromwich Albion'),
 ('Burnley', 'Crystal Palace'),
 ('Manchester City', 'Liver